# PRIZM Data Wrangling Tutorial


Required standard modules:

In [4]:
import numpy as np
import matplotlib.pyplot as plt

## PRIZM Metadatabase

This section demonstrates the basic functionalities of the PRIZM metadatabase. We begin by importing the metadatabase module under the `mdb` alias.

In [4]:
import metadatabase as mdb

### Retrieving Metadata

SQLite queries can be executed against the metadatabase using the `execute` function. For instance, the following construction can be used to retrieve the model number and description of every hardware component listed in the PRIZM metadatabase.

In [ ]:
mdb.execute("SELECT component_model, component_description FROM HardwareComponents")

As an example of a more complex query, the directory addresses and file names associated with the east-west polarization data gathered by the 100MHz PRIZM antenna during the first half of 2018 can be retrieved in chronological order as follows.

In [ ]:
mdb.execute(("SELECT DataDirectories.directory_address, DataTypes.file_name "
             "FROM   DataDirectories "
             "JOIN   DataCategories "
             "ON     DataDirectories.data_category = DataCategories.data_category "
             "AND    DataCategories.category_name = 'Antenna' "
             "JOIN   DataFiles "
             "ON     DataDirectories.data_directory = DataFiles.data_directory "
             "AND    DataDirectories.time_start <= strftime('%s','2018-07-01 00:00:00') "
             "AND    DataDirectories.time_stop >= strftime('%s','2018-01-01 00:00:00') "
             "JOIN   DataTypes "
             "ON     DataFiles.data_file = DataTypes.data_file "
             "JOIN   ChannelGroups "
             "ON     DataFiles.channel_group = ChannelGroups.channel_group "
             "JOIN   ChannelOrientations "
             "ON     ChannelOrientations.channel_orientation = ChannelGroups.channel_orientation "
             "AND    ChannelOrientations.orientation_name = 'EW' "
             "JOIN   ArrayElements "
             "ON     ArrayElements.array_element = ChannelGroups.array_element "
             "AND    ArrayElements.element_name = '100MHz' "
             "ORDER  BY DataDirectories.time_start "))

## PRIZM Data Container

This section demonstrates the basic functionalities of the PRIZM data container. We begin by importing the container object as follows.

In [2]:
from data import Data

### Loading Data via the Metadatabase

PRIZM data can be loaded through the metadatabase using the data container's `via_metadatabase` constructor. This function receives lists as arguments, and returns a data container holding the data matching all combinations of these input lists' elements. This is illustrated below, where absolutely all data collected around April 22–23, 2018 is loaded.

In [ ]:
data = Data.via_metadatabase(categories=['Antenna', 'Switch', 'Temperature'],
                             instruments=['100MHz', '70MHz'],
                             channels=['EW', 'NS'],
                             intervals=[(1524400000.0,1524500000.0),],
                             quality=[1, 0, 'NULL'],
                             integrity=[1, 0, 'NULL'],
                             completeness=[1, 0, 'NULL'])

Alternatively, curated data selections suitable for specific analyses can be loaded through the metadatabase by referencing certain pickle files, such as those available under this repository's `../selections` subdirectory. As demonstrated below, the pickle file `../selections/2018_100MHz_EW.p` can be referenced to load all the good-quality east-west polarization data gathered by the 100MHz antenna in 2018.

In [35]:
data = Data.via_metadatabase(selection='./selections/2018_100MHz_EW.p')

### Loading Data from Directories

PRIZM data can be loaded directly from a list of directories using the data container's `from_directories` constructor. Because this approach does not make use of the metadatabase, it requires a substantial amount of additional information as inputs in order to make sense of the file and directory structure the data will be read from. User-defined catalogues are used for that purpose. Below an example of such catalogues is shown.

In [1]:
classification_catalogue = {
    'data_70MHz': '70MHz',
    'data_100MHz': '100MHz',
}

file_catalogue = {
    'pol0.scio': ('float',['EW'],'pol'),
    'pol0.scio.bz2': ('float',['EW'],'pol'),
    'pol1.scio': ('float',['NS'],'pol'),
    'pol1.scio.bz2': ('float',['NS'],'pol'),
    'time_sys_stop.raw': ('float',['EW','NS'],'time_sys_stop'),
    'time_sys_start.raw': ('float',['EW','NS'],'time_sys_start'),
    'open.scio': ('float',['Switch'],'open'),
    'short.scio': ('float',['Switch'],'short'),
    'res50.scio': ('float',['Switch'],'res50'),
    'res100.scio': ('float',['Switch'],'res100'),
    'antenna.scio': ('float',['Switch'],'antenna'),
    'open.scio.bz2': ('float',['Switch'],'open'),
    'short.scio.bz2': ('float',['Switch'],'short'),
    'res50.scio.bz2': ('float',['Switch'],'res50'),
    'res100.scio.bz2': ('float',['Switch'],'res100'),
    'antenna.scio.bz2': ('float',['Switch'],'antenna'),
}

While the `classification_catalogue` connects parent directory names to the primary data container keys, the `file_catalogue` lists every file of interest along with its respective data type, container hierarchy keys, and container entry name. Notice, however, that the above examples are neither definitive nor exhaustive, and would need to be manually edited to accommodate additional data, different file names, and/or different parent directory names.

In addition to the above catalogues, the `from_directories` constructor also receives a list of directory addresses as an argument, and returns a data container holding the data matching all cataloged files found within every subdirectory of the input directory addresses. This is illustrated below, where some of the data collected by the 100MHz instrument around October 21–22, 2021 is loaded.

In [ ]:
data = Data.from_directories(directory_addresses=['/project/s/sievers/prizm/marion2022/prizm-100/data_100MHz/16348',
                                                  '/project/s/sievers/prizm/marion2022/prizm-100/data_100MHz/switch/16348'],
                             classification_catalogue=classification_catalogue,
                             file_catalogue=file_catalogue)

### Data Manipulation

The PRIZM data container organizes the loaded data in a nested dictionary structure, with each data entry being stored as a `numpy.ndarray`.
```python
{
    '100MHz':
    {
        'EW':
        {
            'pol': numpy.ndarray,
            'time_sys_start': numpy.ndarray,
            ...
        },

        'NS':
        {
            'pol': numpy.ndarray,
            'time_sys_start': numpy.ndarray,
            ...
        },

        'Switch':
        {
            'antenna': numpy.ndarray,
            'short': numpy.ndarray,
            ...
        },

        'Housekeeping':
        {
            'cross_real': numpy.ndarray,
            'temp_100_ambient': numpy.ndarray,
            ...
        },

    '70MHz':
    {
        ...
    }
}
```
The container's primary keys, `100MHz` and `70MHz`, refer to the two PRIZM instruments. The data associated with a particular polarization channel are listed under the appropriate channel key, as examplified by the `pol` and `time_sys_start` entries under both the `EW` and `NS` keys. Meanwhile, the data describing the instrument's switching cadence are listed under the `Switch` key, as illustrated above by the `antenna` and `short` entries. Any remaining data are listed under the `Housekeeping` key, as shown by the `cross_real` and `temp_100_ambient` entries.


#### Accessing Data

Each entry of the data container can be accessed as in a Python dictionary. A few examples are listed below.

In [ ]:
data['100MHz']['EW']['pol']

In [ ]:
data['100MHz']['EW']['time_sys_start']

In [ ]:
data['100MHz']['Switch']['antenna']

#### Computing LST

The `lst` method produces local sidereal time entries from the data's UTC Unix timestamps. These entries are stored as `lst_sys_start` and `lst_sys_stop` under the instrument and channel keys provided by the user.

In [ ]:
data.lst(instruments=['100MHz'], channels=['EW', 'NS'])

In [ ]:
data['100MHz']['EW']['lst_sys_start']

In [ ]:
data['100MHz']['EW']['lst_sys_stop']

In [ ]:
data['100MHz']['NS']['lst_sys_start']

In [ ]:
data['100MHz']['NS']['lst_sys_stop']

#### Data Partitioning

Because each PRIZM instrument regularly switches between performing sky observations and measuring the signal strength of its internal calibration sources, partitioning the data according to this switching cadence becomes an important aspect of data manipulation. Such partitions can be automatically generated with the `partition` method, as demonstrated below. The resulting partitions are stored under the instrument and channel keys provided by the user.

In [ ]:
data.partition(instruments=['100MHz'], channels=['EW', 'NS'], buffer=(1,1))

In [ ]:
data['100MHz']['EW']['Partitions']

In [ ]:
data['100MHz']['NS']['Partitions']

#### Data Slicing

The `get` method can used to extract data associated with a specific partition. As an example, the sky observations and timestamps associated with the east-west channel of the 100MHz antenna can be extracted as follows.

In [ ]:
data.get(data='pol', instrument='100MHz', channel='EW', partition='antenna')

In [ ]:
data.get(data='time_sys_start', instrument='100MHz', channel='EW', partition='antenna')

#### Spectra Interpolation

Spectra associated with a specific data partition can also be extrapolated through linear interpolation with the help of the `interpolate` method. This is illustrated below, where the spectra of a calibrator interpolated for times at which the instrument was actually observing the sky.

In [ ]:
data.interpolate([...,...], instrument='100MHz', channel='EW', partition='short', threshold=500)

### Data Visualization

*Under Construction*

In [ ]:
plt.figure(1, figsize=(10,5))
plt.plot(data['100MHz']['EW']['lst_sys_start'])

In [ ]:
plt.figure(2, figsize=(15,15))
plt.imshow(np.log10(data.get(data='pol', instrument='100MHz', channel='EW', partition='antenna')),
           vmin=4.5, vmax=8.5, aspect=1)

In [ ]:
plt.figure(3, figsize=(10,5))
plt.plot(np.log10(data.get(data='pol', instrument='100MHz', channel='EW', partition='antenna'))[0,:],
         color='black', label='antenna')
plt.plot(np.log10(data.get(data='pol', instrument='100MHz', channel='EW', partition='short'))[0,:],
         color='blue', label='short')
plt.plot(np.log10(data.get(data='pol', instrument='100MHz', channel='EW', partition='noise'))[0,:],
         color='orange', label='noise')
plt.legend()